In [1]:
from typing import List

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table

import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff

import flask
from flask_cors import CORS

import numpy as np
import pandas as pd
from pandas import Series
from pandas import DataFrame

import os
from os import environ
import requests
import io

import datetime
from dateutil.relativedelta import relativedelta

import socket

In [2]:
analyst = environ['analyst']
#analyst='pierrick'
path='/home/ubuntu/Data/'
port={'pierrick': 3999, 'nancy': 4000, 'patrick': 4001, 'testing': 3998}

In [3]:
files_Risk=[x for x in os.listdir(path+'FrontEnd_Input_'+analyst+'/') if 'Risk' in x]
Risk=pd.read_csv(path+'FrontEnd_Input_'+analyst+'/'+files_Risk[0])
for f in files_Risk[1:]:
    Intermediate=pd.read_csv(path+'FrontEnd_Input_'+analyst+'/'+f)
    Risk=pd.concat([Risk,Intermediate])
Risk['Route']=Risk['Route'].astype('str')
Risk['DepDate']=Risk['DepDate'].astype('str')    
Risk['FltNum']=Risk['FltNum'].astype('str')
Risk['dtime']=Risk['dtime'].astype('str')

files_IdealCurve=[x for x in os.listdir(path+'FrontEnd_Input_'+analyst+'/') if 'IdealCurve' in x]
IdealCurve=pd.read_csv(path+'FrontEnd_Input_'+analyst+'/'+files_IdealCurve[0])
for f in files_IdealCurve[1:]:
    Intermediate=pd.read_csv(path+'FrontEnd_Input_'+analyst+'/'+f)
    IdealCurve=pd.concat([IdealCurve,Intermediate])
IdealCurve['Route']=IdealCurve['Route'].astype('str')
IdealCurve['DepDate']=IdealCurve['DepDate'].astype('str')    
IdealCurve['FltNum']=IdealCurve['FltNum'].astype('str')
IdealCurve['dtime']=IdealCurve['dtime'].astype('str')
    
files_Actuals=[x for x in os.listdir(path+'FrontEnd_Input_'+analyst+'/') if 'Actuals' in x]
Actuals=pd.read_csv(path+'FrontEnd_Input_'+analyst+'/'+files_Actuals[0])
for f in files_Actuals[1:]:
    Intermediate=pd.read_csv(path+'FrontEnd_Input_'+analyst+'/'+f)
    Actuals=pd.concat([Actuals,Intermediate])
Actuals['Route']=Actuals['Route'].astype('str')
Actuals['DepDate']=Actuals['DepDate'].astype('str')    
Actuals['FltNum']=Actuals['FltNum'].astype('str')
Actuals['dtime']=Actuals['dtime'].astype('str')
    
files_Capacity=[x for x in os.listdir(path+'FrontEnd_Input_'+analyst+'/') if 'Capacity' in x]
Capacity=pd.read_csv(path+'FrontEnd_Input_'+analyst+'/'+files_Capacity[0])
for f in files_Capacity[1:]:
    Intermediate=pd.read_csv(path+'FrontEnd_Input_'+analyst+'/'+f)
    Capacity=pd.concat([Capacity,Intermediate])
Capacity['Route']=Capacity['Route'].astype('str')
Capacity['DepDate']=Capacity['DepDate'].astype('str')    
Capacity['FltNum']=Capacity['FltNum'].astype('str')
Capacity['dtime']=Capacity['dtime'].astype('str')

files_GroupPax=[x for x in os.listdir(path+'FrontEnd_Input_'+analyst+'/') if 'GroupPax' in x]
GroupPax=pd.read_csv(path+'FrontEnd_Input_'+analyst+'/'+files_GroupPax[0])
for f in files_GroupPax[1:]:
    Intermediate=pd.read_csv(path+'FrontEnd_Input_'+analyst+'/'+f)
    GroupPax=pd.concat([GroupPax,Intermediate])
GroupPax['Route']=GroupPax['Route'].astype('str')
GroupPax['DepDate']=GroupPax['DepDate'].astype('str')    
GroupPax['FltNum']=GroupPax['FltNum'].astype('str')
GroupPax['dtime']=GroupPax['dtime'].astype('str')

GroupPax=GroupPax.rename(columns={'Group_pax_cumul': 'Group Pax'})

files_LACeco=[x for x in os.listdir(path+'FrontEnd_Input_'+analyst+'/') if 'LAC' in x and 'eco' in x]
LACeco=pd.read_csv(path+'FrontEnd_Input_'+analyst+'/'+files_LACeco[0])
for f in files_LACeco[1:]:
    Intermediate=pd.read_csv(path+'FrontEnd_Input_'+analyst+'/'+f)
    LACeco=pd.concat([LACeco,Intermediate])
LACeco['Route']=LACeco['Route'].astype('str')
LACeco['DepDate']=LACeco['DepDate'].astype('str')    
LACeco['FltNum']=LACeco['FltNum'].astype('str')
LACeco['dtime']=LACeco['dtime'].astype('str')

files_LACbusiness=[x for x in os.listdir(path+'FrontEnd_Input_'+analyst+'/') if 'LAC' in x and 'business' in x]
LACbusiness=pd.read_csv(path+'FrontEnd_Input_'+analyst+'/'+files_LACbusiness[0])
for f in files_LACbusiness[1:]:
    Intermediate=pd.read_csv(path+'FrontEnd_Input_'+analyst+'/'+f)
    LACbusiness=pd.concat([LACbusiness,Intermediate])
LACbusiness['Route']=LACbusiness['Route'].astype('str')
LACbusiness['DepDate']=LACbusiness['DepDate'].astype('str')    
LACbusiness['FltNum']=LACbusiness['FltNum'].astype('str')
LACbusiness['dtime']=LACbusiness['dtime'].astype('str')

files_BookingPace=[x for x in os.listdir(path+'FrontEnd_Input_'+analyst+'/') if 'BookingPace' in x]
BookingPace=pd.read_csv(path+'FrontEnd_Input_'+analyst+'/'+files_BookingPace[0])
for f in files_BookingPace[1:]:
    Intermediate=pd.read_csv(path+'FrontEnd_Input_'+analyst+'/'+f)
    BookingPace=pd.concat([BookingPace,Intermediate])
BookingPace['Route']=BookingPace['Route'].astype('str')
BookingPace['DepDate']=BookingPace['DepDate'].astype('str')    
BookingPace['FltNum']=BookingPace['FltNum'].astype('str')
BookingPace['dtime']=BookingPace['dtime'].astype('str')

# Import external stylesheet

In [4]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Vectors for dropdown menues

In [5]:
Mapping_StrToDt=pd.concat([Series(Risk['DepDate'].unique()),Series([pd.to_datetime(x) for x in Risk['DepDate'].unique()])],axis=1)
Mapping_StrToDt.columns=['DepDate','DepDate_new']
Risk=Risk.merge(Mapping_StrToDt,on='DepDate')
Risk['DepDate']=Risk['DepDate_new']
Risk=Risk[[x for x in Risk.columns if x!='DepDate_new']]

#################################################################

Mapping_StrToDt=pd.concat([Series(IdealCurve['DepDate'].unique()),Series([pd.to_datetime(x) for x in IdealCurve['DepDate'].unique()])],axis=1)
Mapping_StrToDt.columns=['DepDate','DepDate_new']
IdealCurve=IdealCurve.merge(Mapping_StrToDt,on='DepDate')
IdealCurve['DepDate']=IdealCurve['DepDate_new']
IdealCurve=IdealCurve[[x for x in IdealCurve.columns if x!='DepDate_new']]

#################################################################

Mapping_StrToDt=pd.concat([Series(Actuals['DepDate'].unique()),Series([pd.to_datetime(x) for x in Actuals['DepDate'].unique()])],axis=1)
Mapping_StrToDt.columns=['DepDate','DepDate_new']
Actuals=Actuals.merge(Mapping_StrToDt,on='DepDate')
Actuals['DepDate']=Actuals['DepDate_new']
Actuals=Actuals[[x for x in Actuals.columns if x!='DepDate_new']]

#################################################################

Mapping_StrToDt=pd.concat([Series(Capacity['DepDate'].unique()),Series([pd.to_datetime(x) for x in Capacity['DepDate'].unique()])],axis=1)
Mapping_StrToDt.columns=['DepDate','DepDate_new']
Capacity=Capacity.merge(Mapping_StrToDt,on='DepDate')
Capacity['DepDate']=Capacity['DepDate_new']
Capacity=Capacity[[x for x in Capacity.columns if x!='DepDate_new']]

#################################################################

Mapping_StrToDt=pd.concat([Series(GroupPax['DepDate'].unique()),Series([pd.to_datetime(x) for x in GroupPax['DepDate'].unique()])],axis=1)
Mapping_StrToDt.columns=['DepDate','DepDate_new']
GroupPax=GroupPax.merge(Mapping_StrToDt,on='DepDate')
GroupPax['DepDate']=GroupPax['DepDate_new']
GroupPax=GroupPax[[x for x in GroupPax.columns if x!='DepDate_new']]

#################################################################

Mapping_StrToDt=pd.concat([Series(BookingPace['DepDate'].unique()),Series([pd.to_datetime(x) for x in BookingPace['DepDate'].unique()])],axis=1)
Mapping_StrToDt.columns=['DepDate','DepDate_new']
BookingPace=BookingPace.merge(Mapping_StrToDt,on='DepDate')
BookingPace['DepDate']=BookingPace['DepDate_new']
BookingPace=BookingPace[[x for x in BookingPace.columns if x!='DepDate_new']]

#################################################################

Mapping_StrToDt=pd.concat([Series(LACeco['DepDate'].unique()),Series([pd.to_datetime(x) for x in LACeco['DepDate'].unique()])],axis=1)
Mapping_StrToDt.columns=['DepDate','DepDate_new']
LACeco=LACeco.merge(Mapping_StrToDt,on='DepDate')
LACeco['DepDate']=LACeco['DepDate_new']
LACeco=LACeco[[x for x in LACeco.columns if x!='DepDate_new']]
LACeco=LACeco.rename(columns={'LAC': 'LAC_eco'})
LACeco=LACeco.drop_duplicates(subset=['Dprio','Route','DepDate','FltNum'])
LACeco=LACeco[[x for x in LACeco.columns if x!='dtime']]

#################################################################

Mapping_StrToDt=pd.concat([Series(LACbusiness['DepDate'].unique()),Series([pd.to_datetime(x) for x in LACbusiness['DepDate'].unique()])],axis=1)
Mapping_StrToDt.columns=['DepDate','DepDate_new']
LACbusiness=LACbusiness.merge(Mapping_StrToDt,on='DepDate')
LACbusiness['DepDate']=LACbusiness['DepDate_new']
LACbusiness=LACbusiness[[x for x in LACbusiness.columns if x!='DepDate_new']]
LACbusiness=LACbusiness.rename(columns={'LAC': 'LAC_bus'})
LACbusiness=LACbusiness.drop_duplicates(subset=['Dprio','Route','DepDate','FltNum'])
LACbusiness=LACbusiness[[x for x in LACbusiness.columns if x!='dtime']]

In [6]:
Risk=Risk.loc[(Risk['DepDate']>=pd.to_datetime('2019-05-21'))&(Risk['DepDate']<=pd.to_datetime('2019-09-30')),:]
IdealCurve=IdealCurve.loc[(IdealCurve['DepDate']>=pd.to_datetime('2019-05-21'))&(IdealCurve['DepDate']<=pd.to_datetime('2019-09-30')),:]
Actuals=Actuals.loc[(Actuals['DepDate']>=pd.to_datetime('2019-05-21'))&(Actuals['DepDate']<=pd.to_datetime('2019-09-30')),:]
GroupPax=GroupPax.loc[(GroupPax['DepDate']>=pd.to_datetime('2019-05-21'))&(GroupPax['DepDate']<=pd.to_datetime('2019-09-30')),:]
Capacity=Capacity.loc[(Capacity['DepDate']>=pd.to_datetime('2019-05-21'))&(Capacity['DepDate']<=pd.to_datetime('2019-09-30')),:]

In [7]:
LACeco['Dtt']=(LACeco['DepDate']-pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d'))).apply(lambda x: np.float(x.days))
LACbusiness['Dtt']=(LACbusiness['DepDate']-pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d'))).apply(lambda x: np.float(x.days))
BookingPace['Dtt']=(BookingPace['DepDate']-pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d'))).apply(lambda x: np.float(x.days))

Risk=Risk.merge(LACeco.loc[LACeco['Dprio']==LACeco['Dtt'],['Route','DepDate','FltNum','Direction','LAC_eco']].drop_duplicates(),
                on=['Route','DepDate','FltNum','Direction'],how='left')

Risk=Risk.merge(LACbusiness.loc[LACbusiness['Dprio']==LACbusiness['Dtt'],['Route','DepDate','FltNum','Direction','LAC_bus']].drop_duplicates(),
                on=['Route','DepDate','FltNum','Direction'],how='left')

Risk=Risk.merge(BookingPace.loc[BookingPace['Dprio']==BookingPace['Dtt'],['Route','DepDate','FltNum',
            'Underpace_7','Underpace_3','Overpace_7','Overpace_3']].drop_duplicates(),
                on=['Route','DepDate','FltNum'],how='left')

Risk=Risk[['Route','DepDate','FltNum','dtime','Direction','month','dday',
     'SpoilageRisk','Underpace_7','Underpace_3','SpillageRisk','Overpace_7','Overpace_3','LAC_eco','LAC_bus']]

In [8]:
Risk['SpoilageRisk']=Risk['SpoilageRisk'].apply(lambda x: np.round(x,2))
Risk['SpillageRisk']=Risk['SpillageRisk'].apply(lambda x: np.round(x,2))
Risk['Underpace_7']=Risk['Underpace_7'].apply(lambda x: np.round(x,2))
Risk['Underpace_3']=Risk['Underpace_3'].apply(lambda x: np.round(x,2))
Risk['Overpace_7']=Risk['Overpace_7'].apply(lambda x: np.round(x,2))
Risk['Overpace_3']=Risk['Overpace_3'].apply(lambda x: np.round(x,2))

Risk=Risk.sort_values(by=['Route','DepDate', 'FltNum', 'dtime', 'Direction'])    

In [ ]:
intermediate=Risk[['Route','DepDate','FltNum','dtime','dday']].drop_duplicates()
checkbox_values=intermediate\
.sort_values(by=['Route','DepDate','FltNum','dtime','dday']).transpose().values.tolist()

In [ ]:
LACeco=LACeco.sort_values(by=['Route', 'DepDate', 'FltNum', 'Dprio'],ascending=[True,True,True,False])

class_vector=['W','R','A','N','G','O','H','V','F','X','L','U','P','S','B','Q','K','M','Y']
class_map=pd.concat([Series(class_vector),
                    Series(range(1,len(class_vector)+1))],axis=1)
class_map.columns=['LAC','Y']

# Layout

In [ ]:
line_columns_Pax = ['Actual Bookings']
linecolor_dict_Pax = dict(zip(line_columns_Pax,['black']))
linedash_dict_Pax = dict(zip(line_columns_Pax,['solid']))
name_dict_Pax = dict(zip(line_columns_Pax,['Actual Bookings']))

line_columns_Ideal = ['Ramp-up frontier', 'Ideal curve (80% LF)', 'Ideal curve (100% LF)', 
                    'Phase-down frontier']
linecolor_dict_Ideal = dict(zip(line_columns_Ideal,['red', 'blue', 'green', '#E0115F']))
linedash_dict_Ideal = dict(zip(line_columns_Ideal,['dot', 'solid', 'solid', 'dot']))
name_dict_Ideal = dict(zip(line_columns_Ideal,['Ramp-up frontier', 'Ideal curve (80% LF)', 
                        'Ideal curve (100% LF)', 'Phase-down frontier']))

line_columns_Capacity = ['Real Cabin Capacity']
linecolor_dict_Capacity = dict(zip(line_columns_Capacity,['lightgrey']))
linedash_dict_Capacity = dict(zip(line_columns_Capacity,['solid']))
name_dict_Capacity = dict(zip(line_columns_Capacity,['Cabin Capacity']))

rename_dict=dict(zip(Risk.columns,
['Route', 'Departure Date', 'Flight Number', 'Departure Time', 'Direction', 'Month', 'Weekday', 
 'Spoilage Risk', 'Underpace (7d)', 'Underpace (3d)', 'Spillage Risk', 'Overpace (7d)', 'Overpace (3d)', 'LAC (eco.)', 'LAC (bus.)']))

line_columns_Lac = ['Y']
linecolor_dict_Lac = dict(zip(line_columns_Lac,['black']))
linedash_dict_Lac = dict(zip(line_columns_Lac,['solid']))
name_dict_Lac = dict(zip(line_columns_Lac,['LAC (eco.)']))

In [ ]:
epoch = datetime.datetime.utcfromtimestamp(0)
def unix_time_millis(dt):
    return int((dt - epoch).total_seconds())#* 1000.0

def get_marks_from_start_end(start, end):
    ''' Returns dict with one item per month
    {1440080188.1900003: '2015-08',
    '''
    result = []
    current = start
    while current <= end:
        result.append(current)
        current += relativedelta(weeks=1)
    return {unix_time_millis(m):(str(m.strftime('%m-%d'))) for m in result}

In [ ]:
PAGE_SIZE = 20

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    
    # MAIN HEADER
    html.Div([html.H4(children='Booking Curve Steering System',
    style={
            'textAlign': 'center',
            'color': 'black',
            'margin-bottom': '0.5em',
            'padding' : '50px' ,
            # 'backgroundColor' : '#66b3cc',
            'width': '20%',
            # 'height': '10%',
            'margin-top': '-2em',
            'margin-left': '-1em'
        }),
    html.Img(src=app.get_asset_url('LuxairGroup_logo.jpg'), 
              style = {'backgroundColor' : '#66b3cc', 
                       'width': '10%', 
                       'height': '8em', 
                       'margin-left': '8em',
                       'margin-top': '-4.25em'
                      # 'margin-right': '1em'
                      }),
        ]),
    
    # ROUTE DROPDOWN AND DATE SLIDER
    
        # DROPDOWN
    html.Div([html.Div(style={'width': '25%', 'margin-bottom': '1em', 'display': 'inline-block'}),
    html.Div([html.Div([html.Label('Route', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_routes',
    options=[],
    placeholder="Select a route",                                 
    value=sorted(set(checkbox_values[0])),
    multi=True)],
    style={'margin-bottom': '2em', 'width': '80%', 'margin-left': '6em'})
    ,
    html.Div([html.Label('Departure Date', style={'textAlign': 'center', 'margin-bottom': '0em', 'font-weight': 'bold'}),
    dcc.RangeSlider(
           id = 'datetime_RangeSlider',
           updatemode = 'mouseup', #don't let it update till mouse released
           min = unix_time_millis(min(checkbox_values[1])),
           max = unix_time_millis(max(checkbox_values[1])),
           value = [unix_time_millis(min(checkbox_values[1])),
                   unix_time_millis(max(checkbox_values[1]))],
           #TODO add markers for key dates
           marks=get_marks_from_start_end(min(checkbox_values[1]),max(checkbox_values[1])),
       )],
    style={'margin-bottom': '2em', 'width': '80%', 'margin-left': '6em'})
    ,
    html.Div([html.Label('Flight Number', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Checklist(id = 'checklist_table_fltnum',
    options=[],
    values=sorted(set(checkbox_values[2])),
    labelStyle = {'display': 'inline', 'cursor': 'pointer', 'margin-left': '0em'})       
             ],   
    style={'margin-bottom': '1em', 'textAlign': 'center'})
    ,
    html.Div([html.Label('Weekday', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Checklist(id = 'checklist_table_dday',
    options=[],
    values=sorted(set(checkbox_values[4])),
    labelStyle = {'display': 'inline', 'cursor': 'pointer', 'margin-left': '0em'}) 
             ],   
    style={'margin-bottom': '1em', 'textAlign': 'center'})
             ], style={'width': '50%', 'margin-bottom': '1em', 'margin-top': '-10em','display': 'inline-block', 'background-color': 'lightgrey'})
    ,
    html.Div(style={'width': '25%', 'margin-bottom': '1em', 'display': 'inline-block'})
    ])
    ,
    # CONTAINERS
    
        # TABLE CONTAINER
    html.Div([dash_table.DataTable(
            id = 'booking_table',
            columns=[{'id': c, 'name': rename_dict[c], 'deletable': True} for c in Risk.columns],
            style_header={'backgroundColor': 'rgb(30, 30, 30)','color': 'white', 'textAlign': 'center'},

             style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(40, 40, 40)',
                'color': 'white',
                'textAlign': 'center'
            },
            {
               'if': {'row_index': 'odd', 'column_id': 'Dprio'},
                 'backgroundColor': '#568dba',
                'color': 'black',
                'textAlign': 'center'
             },    
            {
                'if': {'row_index': 'even'},
                'backgroundColor': 'rgb(50, 50, 50)',
                'color': 'white',
                'textAlign': 'center'
            },
            {
                'if': {'row_index': 'even', 'column_id': 'Dprio'},
                'backgroundColor': '#6798c1',
                'color': 'black',
                 'textAlign': 'center'
            },     
            {
                 'if': {
                     'column_id': 'SpoilageRisk',
                     'filter': 'SpoilageRisk >= num(0.8)',
                 },
                 'color': 'red',
            },
            {
                'if': {
                     'column_id': 'SpillageRisk',
                     'filter': 'SpillageRisk >= num(0.8)',
                 },
                'color': '#E0115F'
            }
            ],
            style_cell={
            # all three widths are needed
            'minWidth': '50px', 'width': '50px', 'maxWidth': '50px',
            'whiteSpace': 'normal'
        },

    pagination_settings={
        'current_page': 0,
        'page_size': PAGE_SIZE
    },        
        
    pagination_mode='be',

    filtering='be',
    filtering_settings='',

    sorting='be',
    sorting_type='multi',
    sorting_settings=[],
    row_selectable="single"

            )
                       ],
        
        style={'width': '52.5%', 'display': 'inline-block', 'margin-top': '1em'})
             
             ,

        # GRAPH CONTAINER                   
#      html.Div(children=[dcc.Graph(id='graph')],
#      style={'width': '47.5%', 'float': 'right', 'display': 'inline-block'},
#              id='graph-container_pax') 
    
        # GRAPH CONTAINER                   
    html.Div([dcc.Tabs(id="tabs", children=[
        dcc.Tab(label='Booking Curve', children=[
            html.Div([
                dcc.Graph(
                    id='graph-bc')
                ])
            ]),
        dcc.Tab(label='LAC', children=[
            html.Div([
                dcc.Graph(
                    id='graph-lac')
                ])
            ])])
    ],style={'width': '47.5%', 'float': 'right', 'display': 'inline-block'}),    
    
    
    html.Div(style={'width': '25%', 'margin-bottom': '2em', 'display': 'inline-block'})
              
])

In [ ]:
@app.callback(
    Output("dropdown_routes", "options"),
    [Input("datetime_RangeSlider", "value"),
    Input("checklist_table_fltnum", "values"),
    Input("checklist_table_dday", "values")]
    )
def set_checklist_route(values_depdate,values_fltnum,values_dday):
    subset=set([checkbox_values[0][i] for i in range(len(checkbox_values[0]))\
    if unix_time_millis(checkbox_values[1][i])>=int(values_depdate[0]) and unix_time_millis(checkbox_values[1][i])<=int(values_depdate[1])\
    if checkbox_values[2][i] in values_fltnum\
    and checkbox_values[4][i] in values_dday])
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values

##########################################################

@app.callback(
    [Output("datetime_RangeSlider", "min"),
    Output("datetime_RangeSlider", "max"),
    Output("datetime_RangeSlider", "marks")],
    [Input("dropdown_routes", "value"),
    Input("checklist_table_fltnum", "values"),
    Input("checklist_table_dday", "values")]
    )
def set_checklist_depdate(values_route,values_fltnum,values_dday):
    subset=set([checkbox_values[1][i] for i in range(len(checkbox_values[0]))\
    if checkbox_values[0][i] in values_route\
    and checkbox_values[2][i] in values_fltnum\
    and checkbox_values[4][i] in values_dday])
    minimum=min([unix_time_millis(x) for x in subset])
    maximum=max([unix_time_millis(x) for x in subset])
    value=[minimum,maximum]
    marks=get_marks_from_start_end([x for x in subset if unix_time_millis(x)==minimum][0],
                                   [x for x in subset if unix_time_millis(x)==maximum][0])
    return minimum, maximum, marks

##########################################################

@app.callback(
    Output("checklist_table_fltnum", "options"),
    [Input("dropdown_routes", "value"),
    Input("datetime_RangeSlider", "value"),
    Input("checklist_table_dday", "values")]
    )
def set_checklist_fltnum(values_route,values_depdate,values_dday):
    subset=set([checkbox_values[2][i] for i in range(len(checkbox_values[0]))\
    if checkbox_values[0][i] in values_route\
    and unix_time_millis(checkbox_values[1][i])>=int(values_depdate[0]) and unix_time_millis(checkbox_values[1][i])<=int(values_depdate[1])\
    and checkbox_values[4][i] in values_dday])
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values

##########################################################

@app.callback(
    Output("checklist_table_dday", "options"),
    [Input("dropdown_routes", "value"),
    Input("datetime_RangeSlider", "value"),
    Input("checklist_table_fltnum", "values")]
    )
def set_checklist_dday(values_route,values_depdate,values_fltnum):
    subset=set([checkbox_values[4][i] for i in range(len(checkbox_values[0]))\
    if checkbox_values[0][i] in values_route\
    and unix_time_millis(checkbox_values[1][i])>=int(values_depdate[0]) and unix_time_millis(checkbox_values[1][i])<=int(values_depdate[1])\
    and checkbox_values[2][i] in values_fltnum])
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values

In [ ]:
@app.callback(Output('booking_table', 'data'),
     #,
    # Output('graph', 'figure')
    [Input('dropdown_routes', 'value'),
     Input('datetime_RangeSlider', 'value'),
     Input('checklist_table_fltnum', 'values'),
     Input('checklist_table_dday', 'values'),
     Input('booking_table', 'pagination_settings'),
     Input('booking_table', 'sorting_settings'),
     Input('booking_table', 'filtering_settings')
     #,
     #Input('booking_table', "derived_virtual_selected_rows")
    ]
    )
def update_table(values_route, values_depdate, values_fltnum, values_dday, 
    pagination_settings, sorting_settings, filtering_settings):
    #, derived_virtual_selected_rows):

#     if derived_virtual_selected_rows is None:
#         derived_virtual_selected_rows = []    
    
    filtering_expressions = filtering_settings.split(' && ')

    dff2 = Risk.loc[Risk['Route'].apply(lambda x: x in values_route)\
                   &Risk['DepDate'].apply(lambda x: unix_time_millis(x)>=values_depdate[0] and unix_time_millis(x)<=values_depdate[1])\
                   &Risk['FltNum'].apply(lambda x: x in values_fltnum)
                   &Risk['dday'].apply(lambda x: x in values_dday),:].copy()   
    
    for filter in filtering_expressions:
        if ' eq ' in filter:
            col_name = filter.split(' eq ')[0].replace("\"","")
            filter_value = filter.split(' eq ')[1]
            dff2 = dff2.loc[dff2[col_name] == filter_value]
        if ' > ' in filter:
            col_name = filter.split(' > ')[0].replace("\"","")
            filter_value = float(filter.split(' > ')[1])
            dff2 = dff2.loc[dff2[col_name] > filter_value]
        if ' < ' in filter:
            col_name = filter.split(' < ')[0].replace("\"","")
            filter_value = float(filter.split(' < ')[1])
            dff2 = dff2.loc[dff2[col_name] < filter_value]

    if len(sorting_settings):
        dff2 = dff2.sort_values(
            [col['column_id'] for col in sorting_settings],
            ascending=[
                col['direction'] == 'asc'
                for col in sorting_settings
            ],
            inplace=False
        )       
       
    return dff2.iloc[
            pagination_settings['current_page']*pagination_settings['page_size']:
            (pagination_settings['current_page'] + 1)*pagination_settings['page_size']
        ].to_dict('records')
#     ,
#     {'data': [], 'layout': go.Layout(
#                  xaxis={'title': 'Days prior to departure', 'range': [0, 365]},
#                  yaxis={'title': 'Number of bookings', 'range': [0, 70]},
#                  height=600)}

In [ ]:
@app.callback(Output('graph-bc', 'figure'),
     #,
    # Output('graph', 'figure')
    [Input('dropdown_routes', 'value'),
     Input('datetime_RangeSlider', 'value'),
     Input('checklist_table_fltnum', 'values'),
     Input('checklist_table_dday', 'values'),
     Input('booking_table', 'pagination_settings'),
     Input('booking_table', 'sorting_settings'),
     Input('booking_table', 'filtering_settings'),
     Input('booking_table', "derived_virtual_selected_rows")
    ]
    )
def update_plot(values_route, values_depdate, values_fltnum, values_dday, 
    pagination_settings, sorting_settings, filtering_settings,derived_virtual_selected_rows):
    
    filtering_expressions = filtering_settings.split(' && ')

    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows = []      
    
    dff2 = Risk.loc[Risk['Route'].apply(lambda x: x in values_route)\
                   &Risk['DepDate'].apply(lambda x: unix_time_millis(x)>=values_depdate[0] and unix_time_millis(x)<=values_depdate[1])\
                   &Risk['FltNum'].apply(lambda x: x in values_fltnum)
                   &Risk['dday'].apply(lambda x: x in values_dday),:].copy()    
    
    for filter in filtering_expressions:
        if ' eq ' in filter:
            col_name = filter.split(' eq ')[0].replace("\"","")
            filter_value = filter.split(' eq ')[1]
            dff2 = dff2.loc[dff2[col_name] == filter_value]
        if ' > ' in filter:
            col_name = filter.split(' > ')[0].replace("\"","")
            filter_value = float(filter.split(' > ')[1])
            dff2 = dff2.loc[dff2[col_name] > filter_value]
        if ' < ' in filter:
            col_name = filter.split(' < ')[0].replace("\"","")
            filter_value = float(filter.split(' < ')[1])
            dff2 = dff2.loc[dff2[col_name] < filter_value]

    if len(sorting_settings):
        dff2 = dff2.sort_values(
            [col['column_id'] for col in sorting_settings],
            ascending=[
                col['direction'] == 'asc'
                for col in sorting_settings
            ],
            inplace=False
        )
        
        
    dff2 = dff2.iloc[
            pagination_settings['current_page']*pagination_settings['page_size']:
            (pagination_settings['current_page'] + 1)*pagination_settings['page_size']
            ]        

    dff2.index=range(dff2.shape[0])
        
         ###########################################################        
        
    if len(derived_virtual_selected_rows)>0:    
    
        selected_route=str(dff2.loc[derived_virtual_selected_rows[0],'Route'])
        selected_depdate=str(dff2.loc[derived_virtual_selected_rows[0],'DepDate'])
        selected_fltnum=str(dff2.loc[derived_virtual_selected_rows[0],'FltNum'])

        dff_plot = Actuals.loc[(Actuals['Route']==selected_route)&(Actuals['DepDate'] == selected_depdate)\
        &(Actuals['FltNum'] == selected_fltnum),:].copy() 
        dff2_plot = IdealCurve.loc[(IdealCurve['Route']==selected_route)&(IdealCurve['DepDate'] == selected_depdate)\
        &(IdealCurve['FltNum'] == selected_fltnum),:].copy() 
        dff3_plot=Capacity.loc[(Capacity['Route']==selected_route)&(Capacity['DepDate'] == selected_depdate)\
            &(Capacity['FltNum'] == selected_fltnum),:].copy()
        dff4_plot=GroupPax.loc[(GroupPax['Route']==selected_route)&(GroupPax['DepDate'] == selected_depdate)\
            &(GroupPax['FltNum'] == selected_fltnum),:].copy()        
        
        #\
        #    .drop_duplicates(subset=['Route','DepDate','FltNum','dtime'])
        
        traces = []
        
        for i in line_columns_Pax:
            traces.append(go.Scatter(
                x=dff_plot['Dprio'].tolist(),
                y=dff_plot[i].tolist(),
                mode = 'lines',
                name = name_dict_Pax[i],
                line = dict(
                  dash = linedash_dict_Pax[i],
                  color = linecolor_dict_Pax[i],
                  width = 2
               )
            ))

        for i in line_columns_Ideal:
            traces.append(go.Scatter(
                x=dff2_plot['Dprio'].tolist(),
                y=dff2_plot[i].tolist(),
                mode = 'lines',
                name = name_dict_Ideal[i],
                line = dict(
                  dash = linedash_dict_Ideal[i],
                  color = linecolor_dict_Ideal[i],
                  width = 2
               )
            ))        

        i='Real Cabin Capacity'
        traces.append(go.Scatter(
                x=dff3_plot['Dprio'].tolist(),
                y=dff3_plot[i].tolist(),
                mode = 'lines',
                name = name_dict_Capacity[i],
                line = dict(
                  dash = linedash_dict_Capacity[i],
                  color = linecolor_dict_Capacity[i],
                  width = 2
               )
        ))
            
        traces.append(go.Scatter(
                x=dff4_plot['Dprio'].tolist(),
                y=dff4_plot['Group Pax'].tolist(),
                fill = 'tozeroy',
                name = 'Group_pax'
            #,
            #    line = dict(
            #      dash = linedash_dict_Capacity[i],
            #      color = linecolor_dict_Capacity[i],
            #      width = 2
            #   )
        ))         
   
        return {'data': traces, 'layout': 
        go.Layout(title=go.layout.Title(
        text='Dep. Date: '+selected_depdate.split(' ')[0]+' / Flt. Number: '+str(selected_fltnum),
        xref='paper',
        x=0.5),
        xaxis={'title': 'Days prior to departure', 'range': [0, 365]},
        yaxis={'title': 'Number of bookings', 'range': [0, dff3_plot['Initial Cabin Capacity'].max()+5]},
        height=600)}
    else:
#         return {'data': [], 'layout': 
#         go.Layout(title=go.layout.Title(
#         text='Plot Title',
#         xref='paper',
#         x=0),
#         xaxis={'title': 'Days prior to departure', 'range': [0, 365]},
#         yaxis={'title': 'Number of bookings', 'range': [0, 70]},
#         height=600)}
    
        return {'data': []}     

In [ ]:
@app.callback(Output('graph-lac', 'figure'),
     #,
    # Output('graph', 'figure')
    [Input('dropdown_routes', 'value'),
     Input('datetime_RangeSlider', 'value'),
     Input('checklist_table_fltnum', 'values'),
     Input('checklist_table_dday', 'values'),
     Input('booking_table', 'pagination_settings'),
     Input('booking_table', 'sorting_settings'),
     Input('booking_table', 'filtering_settings'),
     Input('booking_table', "derived_virtual_selected_rows")
    ]
    )
def update_plot(values_route, values_depdate, values_fltnum, values_dday, 
    pagination_settings, sorting_settings, filtering_settings,derived_virtual_selected_rows):
    
    filtering_expressions = filtering_settings.split(' && ')

    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows = []      
    
    dff2 = Risk.loc[Risk['Route'].apply(lambda x: x in values_route)\
                   &Risk['DepDate'].apply(lambda x: unix_time_millis(x)>=values_depdate[0] and unix_time_millis(x)<=values_depdate[1])\
                   &Risk['FltNum'].apply(lambda x: x in values_fltnum)
                   &Risk['dday'].apply(lambda x: x in values_dday),:].copy()    
    
    for filter in filtering_expressions:
        if ' eq ' in filter:
            col_name = filter.split(' eq ')[0].replace("\"","")
            filter_value = filter.split(' eq ')[1]
            dff2 = dff2.loc[dff2[col_name] == filter_value]
        if ' > ' in filter:
            col_name = filter.split(' > ')[0].replace("\"","")
            filter_value = float(filter.split(' > ')[1])
            dff2 = dff2.loc[dff2[col_name] > filter_value]
        if ' < ' in filter:
            col_name = filter.split(' < ')[0].replace("\"","")
            filter_value = float(filter.split(' < ')[1])
            dff2 = dff2.loc[dff2[col_name] < filter_value]

    if len(sorting_settings):
        dff2 = dff2.sort_values(
            [col['column_id'] for col in sorting_settings],
            ascending=[
                col['direction'] == 'asc'
                for col in sorting_settings
            ],
            inplace=False
        )
        
        
    dff2 = dff2.iloc[
            pagination_settings['current_page']*pagination_settings['page_size']:
            (pagination_settings['current_page'] + 1)*pagination_settings['page_size']
            ]        

    dff2.index=range(dff2.shape[0])
        
         ###########################################################        
        
    if len(derived_virtual_selected_rows)>0:    
    
        selected_route=str(dff2.loc[derived_virtual_selected_rows[0],'Route'])
        selected_depdate=str(dff2.loc[derived_virtual_selected_rows[0],'DepDate'])
        selected_fltnum=str(dff2.loc[derived_virtual_selected_rows[0],'FltNum'])

        dff_plot = LACeco.loc[(LACeco['Route']==selected_route)&(LACeco['DepDate'] == selected_depdate)\
        &(LACeco['FltNum'] == selected_fltnum),:].copy()       
        
        #\
        #    .drop_duplicates(subset=['Route','DepDate','FltNum','dtime'])
        
        traces = []
        
        for i in line_columns_Lac:
            traces.append(go.Scatter(
                x=dff_plot['Dprio'].tolist(),
                y=dff_plot[i].tolist(),
                mode = 'lines',
                name = name_dict_Lac[i],
                line = dict(
                  dash = linedash_dict_Lac[i],
                  color = linecolor_dict_Lac[i],
                  width = 2
               )
            ))
   
        return {'data': traces, 'layout': 
        go.Layout(title=go.layout.Title(
        text='Dep. Date: '+selected_depdate.split(' ')[0]+' / Flt. Number: '+str(selected_fltnum),
        xref='paper',
        x=0.5),
        xaxis={'title': 'Days prior to departure', 'range': [0, 365]},
        yaxis={'title': 'LAC', 'range': [class_map['Y'].min(),class_map['Y'].max()],
               'tickvals': class_map['Y'].tolist(), 'ticktext': class_map['LAC'].tolist()},
        height=600)}
    else:
#         return {'data': [], 'layout': 
#         go.Layout(title=go.layout.Title(
#         text='Plot Title',
#         xref='paper',
#         x=0),
#         xaxis={'title': 'Days prior to departure', 'range': [0, 365]},
#         yaxis={'title': 'Number of bookings', 'range': [0, 70]},
#         height=600)}
    
        return {'data': []}     

In [ ]:
# def run_server(self,
#                port=8050,
#                debug=True,
#                threaded=True,
#                **flask_run_options):
#     self.server.run(port=port, debug=debug, **flask_run_options)

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False,host='10.0.0.3',port=port[analyst])

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://10.0.0.3:3999/ (Press CTRL+C to quit)
172.27.2.85 - - [24/May/2019 17:59:44] "GET / HTTP/1.1" 200 -
172.27.2.85 - - [24/May/2019 17:59:44] "GET /_dash-dependencies HTTP/1.1" 200 -
172.27.2.85 - - [24/May/2019 17:59:44] "GET /_dash-layout HTTP/1.1" 200 -
172.27.2.85 - - [24/May/2019 17:59:45] "POST /_dash-update-component HTTP/1.1" 200 -
172.27.2.85 - - [24/May/2019 17:59:45] "POST /_dash-update-component HTTP/1.1" 200 -
172.27.2.85 - - [24/May/2019 17:59:45] "POST /_dash-update-component HTTP/1.1" 200 -
172.27.2.85 - - [24/May/2019 17:59:45] "POST /_dash-update-component HTTP/1.1" 200 -
172.27.2.85 - - [24/May/2019 17:59:45] "POST /_dash-update-component HTTP/1.1" 200 -
172.27.2.85 - - [24/May/2019 17:59:45] "POST /_dash-update-component HTTP/1.1" 200 -
172.27.2.85 - - [24/May/2019 17:59:45] "POST /_dash-update-component HTTP/1.1" 200 -
172.27.2.85 - - [24/May/2019 17:59:45] "POST /_dash-update-component HTTP/1.1" 200 -
172.27.2.85 - - [24/May/2019 17:59:45] "POST 